<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 10. Временные ряды. Блендинг моделей.</center>
## <center>Практика.  Смешивание случайного леса и градиентного бустинга для набора данных по оттоку клиентов телеком-компании</center>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
import sys
sys.path.append('../../scripts/')
from log_progress import log_progress
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Считываем данные.**

In [2]:
df = pd.read_csv('../../data/telecom_churn.csv')

In [3]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [4]:
y = df['Churn']
df.drop(['State', 'Churn'], axis=1, inplace=True)

**Преобразуем строковые Yes/No  признаки в бинарные.**

In [5]:
df['International plan'] = (df['International plan'] == 'Yes').astype('int')
df['Voice mail plan'] = (df['Voice mail plan'] == 'Yes').astype('int')

**Определим важность признаков с помощью случайного леса.**

In [6]:
rf = RandomForestClassifier(random_state=1).fit(df, y)

In [7]:
feaure_imp_df = pd.DataFrame(rf.feature_importances_,
             index=df.columns).sort([0], ascending=False)
feaure_imp_df

,0
Total day minutes,0.166285
Customer service calls,0.120678
Total day charge,0.105372
International plan,0.076782
Total eve charge,0.075501
Total eve minutes,0.065616
Total intl calls,0.059661
Total intl charge,0.046290
Total intl minutes,0.041530
Total night charge,0.040650


**Будем обучать лес на четных признаках, а Xgboost - на нечетных (в порядке убывания важности). Это именно для демонстрации эффективности смешивания алгоритмов. В реальной задаче желательно создать много-много признаков и разыне модели обучать на разных подмножествах признаков.**

In [8]:
rf_features = feaure_imp_df.index.tolist()[::2]
xgb_features = feaure_imp_df.index.tolist()[1::2]

**Отделим 30% данных, чтоб в конце проверять настроенные модели.**

In [9]:
X_train, X_holdout, y_train, y_holdout = \
                                train_test_split(df, y, test_size=0.3,
                                                 random_state=42)

### Кросс-валидация

**Реализуйте функцию, которая возвращает прогнозы модели (в данном случае - вероятности оттока) в процессе кросс-валидации.**

In [10]:
def out_of_fold_prediction(clf, train_df, y_series, folds):
    
    prediction = y_series.copy()
    for train_idx, test_idx in folds:
        X_train, X_test = # Ваш код здесь
        y_train, y_test = # Ваш код здесь
        
        clf.fit(X_train, y_train)
        
        prediction.iloc[test_idx] = # Ваш код здесь
        
    return(prediction)

In [11]:
folds = StratifiedKFold(y_train, n_folds=5, random_state=42)

**Out-of-Fold-прогноз RF с признаками rf_features. Подберите глубину деревьев. Используйте 100 деревьев для каждого леса и параметр random_state=1.**

In [ ]:
%%time
depths = range(13, 20)
scores = []

for depth in log_progress(depths):
    pred = out_of_fold_prediction # Ваш код здесь
    scores.append(roc_auc_score(y_train, pred))

plot(depths, scores)
print(max(scores), depths[np.argmax(scores)])

** Сравните ROC AUC RandomForest на CV и на оставленной выборке.**

In [ ]:
%%time
forest = RandomForestClassifier # Ваш код здесь
forest.fit(X_train, y_train)
forest_oof_pred = out_of_fold_prediction # Ваш код здесь
forest_ho_pred = forest.predict_proba # Ваш код здесь
print('CV: {0}, Holdout: {1}'.format(roc_auc_score(y_train, forest_oof_pred),
                                     roc_auc_score(y_holdout, forest_ho_pred)))

**Out-of-Fold-прогноз Xgboost с признаками xgb_features. Подберите глубину деревьев. Используйте 100 итераций бустинга.**

In [ ]:
%%time

depths = range(2, 13)
scores = []

for depth in log_progress(depths):
    pred = out_of_fold_prediction # Ваш код здесь
    scores.append(roc_auc_score(y_train, pred))

plot(depths, scores)
print(max(scores), depths[np.argmax(scores)])

** Сравните ROC AUC Xgboost на CV и на оставленной выборке.**

In [ ]:
%%time
xgb = XGBClassifier # Ваш код здесь
xgb.fit(X_train, y_train)
xgb_oof_pred = out_of_fold_prediction # Ваш код здесь
xgb_ho_pred = xgb.predict_proba # Ваш код здесь
print('CV: {0}, Holdout: {1}'.format(roc_auc_score(y_train, xgb_oof_pred),
                                     roc_auc_score(y_holdout, xgb_ho_pred)))

### Смешивание Xgboost и RF

In [ ]:
def best_linear_mix(target, pred1, pred2, metric=roc_auc_score):
    best_metric, best_w1 = 0, 0
    for w1 in np.linspace(0, 1, 20):
        curr_pred = (w1 * pred1 + (1 - w1) * pred2)
        curr_metric = metric(target, curr_pred)
        if curr_metric > best_metric:
            best_metric = curr_metric
            best_pred = curr_pred
            best_w1 = w1
    return best_metric, best_pred, best_w1

In [ ]:
best_auc, best_pred, best_w1 = best_linear_mix(y_train, 
                                        xgb_oof_pred, forest_oof_pred)

In [ ]:
best_auc, best_w1

**Определите ROC AUC смеси моделей с найденным весом на оставленной выборке.**

In [ ]:
xgb_rf_ho_pred = # Ваш код здесь
print(roc_auc_score(y_holdout, xgb_rf_ho_pred)